<a href="https://colab.research.google.com/github/Sreyas2403/StockAnalysis_Patterns-Indicators/blob/main/Sector%26MarketCap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os

# Define the directory containing the CSV files
csv_directory = '/content/Data'  # Update this path as your file

# List of CSV files for different sectors and industries
sector_industry_files = [
    'ind_nifty_privatebanklist.csv', 'ind_niftyautolist.csv', 'ind_niftybanklist.csv',
    'ind_niftyconsumerdurableslist.csv', 'ind_niftyfinancelist.csv', 'ind_niftyfmcglist.csv',
    'ind_niftyhealthcarelist.csv', 'ind_niftyitlist.csv', 'ind_niftymedialist.csv',
    'ind_niftymetallist.csv', 'ind_niftyoilgaslist.csv', 'ind_niftypharmalist.csv',
    'ind_niftypsubanklist.csv', 'ind_niftyrealtylist.csv'
]


# Directory containing theme sector files
theme_sector_directory = '/content/ThemeData'  # Update this path with your actual theme sector directory

nifty50_file = '/content/ind_nifty50list.csv'
nifty100_file = '/content/ind_nifty100list.csv'
nifty500_file = '/content/ind_nifty500list.csv'
medium_cap_file = '/content/ind_niftymidcap150list.csv'
small_cap_file = '/content/ind_niftysmallcap250list.csv'


# Read Nifty 500 CSV file
nifty500_df = pd.read_csv(os.path.join(csv_directory, nifty500_file))
nifty500_symbols = nifty500_df['Symbol'].tolist()

# Read Cap CSV files
medium_cap_df = pd.read_csv(os.path.join(csv_directory, medium_cap_file))
small_cap_df = pd.read_csv(os.path.join(csv_directory, small_cap_file))

# Read Nifty50 and Nifty100 CSV files
nifty50_df = pd.read_csv(os.path.join(csv_directory, nifty50_file))
nifty100_df = pd.read_csv(os.path.join(csv_directory, nifty100_file))
nifty50_symbols = nifty50_df['Symbol'].tolist()
nifty100_symbols = nifty100_df['Symbol'].tolist()

# Read theme sector files
theme_sector_data = {}
for file in os.listdir(theme_sector_directory):
    if file.endswith('.csv'):
        theme_df = pd.read_csv(os.path.join(theme_sector_directory, file))
        theme_sector = os.path.splitext(file)[0]  # Get the theme sector name from the file name
        for _, row in theme_df.iterrows():
            symbol = row['Symbol']
            industry = row['Industry']
            theme_sector_data[symbol] = (industry, theme_sector)

# Function to determine cap
def determine_cap(symbol):
    if symbol in medium_cap_df['Symbol'].tolist():
        return 'Medium'
    elif symbol in small_cap_df['Symbol'].tolist():
        return 'Small'
    else:
        return 'None'

# Consolidate data
consolidated_data = []

for file in sector_industry_files:
    sector_industry_df = pd.read_csv(os.path.join(csv_directory, file))
    industry = sector_industry_df['Industry'][0]  # Assuming the industry is the same for all rows in this file
    sector = os.path.splitext(file)[0]  # Get the sector name from the file name

    for symbol in sector_industry_df['Symbol']:
        nifty500_status = 'True' if symbol in nifty500_symbols else 'False'
        cap = determine_cap(symbol)
        nifty50_status = 'True' if symbol in nifty50_symbols else 'False'
        nifty100_status = 'True' if symbol in nifty100_symbols else 'False'

        theme_industry, theme_sector = theme_sector_data.get(symbol, ('None', 'None'))

        consolidated_data.append({
            'Symbol': symbol,
            'Nifty50': nifty50_status,
            'Nifty100': nifty100_status,
            'Nifty 500': nifty500_status,
            'Industry': industry,
            'Cap': cap,
            'Sector': sector,
            'Theme_Industry': theme_industry,
            'Theme_Sector': theme_sector
        })

# Create a DataFrame from the consolidated data
consolidated_df = pd.DataFrame(consolidated_data)

# Save the consolidated DataFrame to an Excel file
output_file = 'finalconsolidated_data.xlsx'
consolidated_df.to_excel(output_file, index=False)

print("Consolidated data has been saved to", output_file)


In [ ]:
import pandas as pd
from nsepython import nse_eq
import yfinance as yf

# Load the Excel file
file_path = '/content/finalconsolidated_data.xlsx'  # Update this to your  file path
df = pd.read_excel(file_path)

# Ensure the 'Symbol' column is of string type
df['Symbol'] = df['Symbol'].astype(str)

# Function to convert NSE symbols to Yahoo Finance symbols
def convert_symbol_to_yf(symbol):
    return symbol + '.NS'

# Convert symbols
df['YF Symbol'] = df['Symbol'].apply(convert_symbol_to_yf)

# Function to get market cap from Yahoo Finance
def get_market_cap(symbol):
    try:
        stock = yf.Ticker(symbol)
        market_cap = stock.info['marketCap']
        return market_cap
    except KeyError:
        print(f"KeyError: Market cap not found for {symbol}")
        return None
    except Exception as e:
        print(f"Error fetching market cap for {symbol}: {e}")
        return None

# Get market caps for all converted symbols
df['Market Cap'] = df['YF Symbol'].apply(get_market_cap)

# Save the results back to an Excel file
output_file_path = 'marketcap_output_file.xlsx'  # Update this to your desired output file path
df.to_excel(output_file_path, index=False)

print("Market capitalization calculation completed. Results saved to", output_file_path)
